In [10]:
import dgl
from dgl.data.utils import load_graphs

import importlib

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import model.model
importlib.reload(model.model)
from model.model import HTGNN, LinkPredictor

from utils.pytorchtools import EarlyStopping
from utils.utils import compute_metric, compute_loss

import utils
import utils.data
importlib.reload(utils)
importlib.reload(utils.data)
from utils.data import load_MAG_data,load_MAG_data_acled,load_MAG_data_acled_time_split, generate_APA_acled, construct_htg_label_mag_acled, load_MAG_data_acled_general




dgl.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

from tqdm import tqdm

In [15]:
import statistics

def evaluate(model, val_feats, val_labels):
    device = torch.device('cuda:0')
    val_mae_list, val_rmse_list = [], []
    auc_list, ap_list, acc_list, f1_list = [], [], [], []
    model.eval()
    with torch.no_grad():
        for (G_feat, (pos_label, neg_label)) in zip(val_feats, val_labels):
            # pdb.set_trace()
            G_feat = G_feat.to(device)
            pos_label = pos_label.to(device)
            neg_label = neg_label.to(device)

            # h = model[0](G_feat, 'author')
            h = model[0](G_feat, 'actor')
            pos_score = model[1](pos_label, h)
            neg_score = model[1](neg_label, h)
            # print('score')
            # print(neg_score)
            loss = compute_loss(pos_score, neg_score, device)
            auc, ap, acc, f1 = compute_metric(pos_score, neg_score)
            auc_list.append(auc)
            ap_list.append(ap)
            acc_list.append(acc)
            f1_list.append(f1)
    
    return auc, ap, statistics.mean(auc_list), statistics.mean(ap_list), statistics.mean(acc_list), statistics.mean(f1_list)

In [4]:
glist, label_dict = load_graphs('data/ogbn_graphs.bin')
glist

[Graph(num_nodes={'author': 17764, 'field_of_study': 14635, 'institution': 2276, 'paper': 27112},
       num_edges={('author', 'affiliated_with', 'institution'): 40307, ('author', 'writes', 'paper'): 101109, ('paper', 'cites', 'paper'): 19926, ('paper', 'has_topic', 'field_of_study'): 285074},
       metagraph=[('author', 'institution', 'affiliated_with'), ('author', 'paper', 'writes'), ('paper', 'paper', 'cites'), ('paper', 'field_of_study', 'has_topic')]),
 Graph(num_nodes={'author': 17764, 'field_of_study': 15186, 'institution': 2276, 'paper': 28584},
       num_edges={('author', 'affiliated_with', 'institution'): 40307, ('author', 'writes', 'paper'): 130156, ('paper', 'cites', 'paper'): 21276, ('paper', 'has_topic', 'field_of_study'): 299817},
       metagraph=[('author', 'institution', 'affiliated_with'), ('author', 'paper', 'writes'), ('paper', 'paper', 'cites'), ('paper', 'field_of_study', 'has_topic')]),
 Graph(num_nodes={'author': 17764, 'field_of_study': 15709, 'institution':

In [5]:
glist, label_dict = load_graphs('graph_data_acled.bin')
glist

[Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('actor', 'involved_in', 'action'): 420, ('actor', 'involved_with', 'actor'): 460},
       metagraph=[('actor', 'action', 'involved_in'), ('actor', 'actor', 'involved_with')]),
 Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('actor', 'involved_in', 'action'): 493, ('actor', 'involved_with', 'actor'): 514},
       metagraph=[('actor', 'action', 'involved_in'), ('actor', 'actor', 'involved_with')]),
 Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('actor', 'involved_in', 'action'): 559, ('actor', 'involved_with', 'actor'): 594},
       metagraph=[('actor', 'action', 'involved_in'), ('actor', 'actor', 'involved_with')]),
 Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('actor', 'involved_in', 'action'): 538, ('actor', 'involved_with', 'actor'): 562},
       metagraph=[('actor', 'action', 'involved_in'), ('actor', 'actor', 'involved_with')]),
 Graph(num_nodes={'actio

In [6]:
glist[0].ndata['_ID']['actor']

tensor([    0,     1,     2,  ..., 18619, 18620, 18621])

In [16]:
glist, label_dict = load_graphs('graph_data_acled.bin')
# glist, label_dict = load_graphs('data/ogbn_graphs.bin')

device = torch.device('cuda:0')
time_window = 2

load_MAG_data_acled(glist[:5], time_window, device)

loading mp2vec
generating train, val, test sets 


([Graph(num_nodes={'action': 25, 'actor': 18622},
        num_edges={('action', 'involved_in_r_t0', 'actor'): 420, ('action', 'involved_in_r_t1', 'actor'): 493, ('actor', 'involved_in_t0', 'action'): 420, ('actor', 'involved_in_t1', 'action'): 493, ('actor', 'involved_with_r_t0', 'actor'): 460, ('actor', 'involved_with_r_t1', 'actor'): 514, ('actor', 'involved_with_t0', 'actor'): 460, ('actor', 'involved_with_t1', 'actor'): 514},
        metagraph=[('action', 'actor', 'involved_in_r_t0'), ('action', 'actor', 'involved_in_r_t1'), ('actor', 'action', 'involved_in_t0'), ('actor', 'action', 'involved_in_t1'), ('actor', 'actor', 'involved_with_r_t0'), ('actor', 'actor', 'involved_with_r_t1'), ('actor', 'actor', 'involved_with_t0'), ('actor', 'actor', 'involved_with_t1')])],
 [(Graph(num_nodes=18622, num_edges=208,
         ndata_schemes={}
         edata_schemes={}),
   Graph(num_nodes=18622, num_edges=208,
         ndata_schemes={}
         edata_schemes={}))],
 [Graph(num_nodes={'action':

In [10]:
len(glist)

1

In [34]:
glist[0].ndata['_ID']['actor']


tensor([    0,     1,     2,  ..., 18619, 18620, 18621])

In [68]:
len(glist[0].in_edges(glist[0].nodes('paper'), etype='writes')[0])
unique_elements, counts = torch.unique(glist[0].in_edges(glist[0].nodes('paper'), etype='writes')[0], return_counts=True)
len(unique_elements)

17764

In [84]:
unique_elements

tensor([    0,     1,     2,  ..., 17761, 17762, 17763])

In [78]:
counts

tensor([ 5, 28,  4,  ...,  1, 15,  5])

In [80]:
num_list = torch.arange(17764)
appearances = [1 if element in unique_elements else 0 for element in num_list]
# all_at_most_once = all(count <= 1 for count in appearances.values())
# all_at_most_once
all(appearances)

True

In [63]:
glist[0].in_edges(glist[0].nodes('action'), etype='involved_in')

(tensor([ 7559,   456, 15077, 15482, 12199, 10728, 15151, 11295,  1760, 13737,
         12193,  3594]),
 tensor([ 2,  5,  6,  6,  7,  9, 10, 12, 18, 20, 21, 24]))

In [55]:
glist[0].ndata['_ID']['author']

tensor([    0,     1,     2,  ..., 17761, 17762, 17763])

In [61]:
glist[0].ndata['_ID']['paper']
unique_elements, counts = torch.unique(glist[0].ndata['_ID']['paper'], return_counts=True)
has_repeats = (counts > 1).any()
has_repeats

tensor(False)

In [52]:
glist[0].ndata['_ID']['actor']

tensor([    0,     1,     2,  ..., 18619, 18620, 18621])

In [53]:
glist[0].ndata['_ID']['action']


tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])

In [ ]:
glist[0].edata

In [99]:
device = torch.device('cuda:0')
time_window = 2

load_MAG_data_acled(glist, time_window, device)

loading mp2vec
a_17764
a_17765
a_17766
a_17767
a_17768
a_17769
a_17770
a_17771
a_17772
a_17773
a_17774
a_17775
a_17776
a_17777
a_17778
a_17779
a_17780
a_17781
a_17782
a_17783
a_17784
a_17785
a_17786
a_17787
a_17788
a_17789
a_17790
a_17791
a_17792
a_17793
a_17794
a_17795
a_17796
a_17797
a_17798
a_17799
a_17800
a_17801
a_17802
a_17803
a_17804
a_17805
a_17806
a_17807
a_17808
a_17809
a_17810
a_17811
a_17812
a_17813
a_17814
a_17815
a_17816
a_17817
a_17818
a_17819
a_17820
a_17821
a_17822
a_17823
a_17824
a_17825
a_17826
a_17827
a_17828
a_17829
a_17830
a_17831
a_17832
a_17833
a_17834
a_17835
a_17836
a_17837
a_17838
a_17839
a_17840
a_17841
a_17842
a_17843
a_17844
a_17845
a_17846
a_17847
a_17848
a_17849
a_17850
a_17851
a_17852
a_17853
a_17854
a_17855
a_17856
a_17857
a_17858
a_17859
a_17860
a_17861
a_17862
a_17863
a_17864
a_17865
a_17866
a_17867
a_17868
a_17869
a_17870
a_17871
a_17872
a_17873
a_17874
a_17875
a_17876
a_17877
a_17878
a_17879
a_17880
a_17881
a_17882
a_17883
a_17884
a_17885
a_17886
a

KeyboardInterrupt: 

In [5]:
glist, label_dict = load_graphs('data/ogbn_graphs.bin')
device = torch.device('cuda:0')
time_window = 2

train_feats, train_labels, val_feats, val_labels, test_feats, test_labels = load_MAG_data(glist, time_window, device)

loading mp2vec
generating train, val, test sets 


KeyError: 'actor'

In [ ]:
glist[0].nodes['author']

In [ ]:
train_feats

In [ ]:
val_feats

In [17]:
glist[:10]

[Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('actor', 'involved_in', 'action'): 420, ('actor', 'involved_with', 'actor'): 460},
       metagraph=[('actor', 'action', 'involved_in'), ('actor', 'actor', 'involved_with')]),
 Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('actor', 'involved_in', 'action'): 493, ('actor', 'involved_with', 'actor'): 514},
       metagraph=[('actor', 'action', 'involved_in'), ('actor', 'actor', 'involved_with')]),
 Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('actor', 'involved_in', 'action'): 559, ('actor', 'involved_with', 'actor'): 594},
       metagraph=[('actor', 'action', 'involved_in'), ('actor', 'actor', 'involved_with')]),
 Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('actor', 'involved_in', 'action'): 538, ('actor', 'involved_with', 'actor'): 562},
       metagraph=[('actor', 'action', 'involved_in'), ('actor', 'actor', 'involved_with')]),
 Graph(num_nodes={'actio

In [140]:
train_feats[0]

Graph(num_nodes={'actor1': 14328, 'actor2': 14328},
      num_edges={('actor1', 'involved_with_t0', 'actor2'): 832, ('actor1', 'involved_with_t1', 'actor2'): 3176, ('actor2', 'involved_with_r_t0', 'actor1'): 832, ('actor2', 'involved_with_r_t1', 'actor1'): 3176},
      metagraph=[('actor1', 'actor2', 'involved_with_t0'), ('actor1', 'actor2', 'involved_with_t1'), ('actor2', 'actor1', 'involved_with_r_t0'), ('actor2', 'actor1', 'involved_with_r_t1')])

In [139]:
train_labels

[(Graph(num_nodes=18622, num_edges=4209,
        ndata_schemes={}
        edata_schemes={}),
  Graph(num_nodes=18622, num_edges=4209,
        ndata_schemes={}
        edata_schemes={}))]

In [120]:
glist, label_dict = load_graphs('graph_data_acled.bin')
time_window = 2
device = torch.device('cuda:0')
train_feats, train_labels, val_feats, val_labels, test_feats, test_labels = load_MAG_data_acled_time_split(glist[:7], 2, 3, device)

loading mp2vec
generating train, val, test sets 


In [24]:
x = [i for i in range(1997,2025)]
y = [i for i in range(len(x))]
dict_year_int = dict(zip(x,y))
dict_ind_year = dict(zip(y,x))

In [25]:
for train_start in range(1997, 2018, 4):
    train_end = train_start+3 if train_start+3<2018 else 2017
    if train_start == train_end:continue
    for val_start in range(train_end+1, 2022, 4):
        val_end = val_start+3 if val_start+3<2022 else 2021
        if val_start == val_end:continue
        test_start = val_end+1 if val_end+1<2025 else 2024
        test_end = 2024
        if test_start == test_end:continue
        print(train_start, train_end, val_start, val_end, test_start, test_end)

1997 2000 2001 2004 2005 2024
1997 2000 2005 2008 2009 2024
1997 2000 2009 2012 2013 2024
1997 2000 2013 2016 2017 2024
1997 2000 2017 2020 2021 2024
2001 2004 2005 2008 2009 2024
2001 2004 2009 2012 2013 2024
2001 2004 2013 2016 2017 2024
2001 2004 2017 2020 2021 2024
2005 2008 2009 2012 2013 2024
2005 2008 2013 2016 2017 2024
2005 2008 2017 2020 2021 2024
2009 2012 2013 2016 2017 2024
2009 2012 2017 2020 2021 2024
2013 2016 2017 2020 2021 2024


In [26]:
import utils
import utils.data
importlib.reload(utils)
importlib.reload(utils.data)
from utils.data import load_MAG_data,load_MAG_data_acled,load_MAG_data_acled_time_split, generate_APA_acled, construct_htg_label_mag_acled, load_MAG_data_acled_general
import os


glist, label_dict = load_graphs('graph_data_acled.bin')
device = torch.device('cuda:0')
time_window = 1

folder_path = "splits/train_1997-2000_valid_2001-2002_test_2003-2005"

for train_start in range(1997, 2018, 4):
    train_end = train_start+3 if train_start+3<2018 else 2017
    if train_start == train_end:continue
    for val_start in range(train_end+1, 2022, 4):
        val_end = val_start+3 if val_start+3<2022 else 2021
        if val_start == val_end:continue
        test_start = val_end+1 if val_end+1<2025 else 2024
        test_end = 2024
        if test_start == test_end:continue
        print(train_start, train_end, val_start, val_end, test_start, test_end)
        print(f"creating splits train_{dict_ind_year[train_start]}-{dict_ind_year[train_end]}_valid_{dict_ind_year[val_start]}-{dict_ind_year[val_end]}_test_{dict_ind_year[test_start]}-{dict_ind_year[test_end]}")
        
        train_feats, train_labels = load_MAG_data_acled_general(glist, dict_ind_year[train_start], dict_ind_year[train_end], time_window, device)
        val_feats, val_labels = load_MAG_data_acled_general(glist, dict_ind_year[val_start], dict_ind_year[val_end], time_window, device)
        test_feats, test_labels = load_MAG_data_acled_general(glist, dict_ind_year[test_start], dict_ind_year[test_end], time_window, device)

        current_folder = f"splits/train_{dict_ind_year[train_start]}-{dict_ind_year[train_end]}_valid_{dict_ind_year[val_start]}-{dict_ind_year[val_end]}_test_{dict_ind_year[test_start]}-{dict_ind_year[test_end]}"
        os.makedirs(current_folder, exist_ok=True)

        torch.save(train_feats, current_folder + '/train_feats.pt')
        torch.save(train_labels, current_folder + '/train_labels.pt')
        torch.save(val_feats, current_folder + '/val_feats.pt')
        torch.save(val_labels, current_folder + '/val_labels.pt')
        torch.save(test_feats, current_folder + '/test_feats.pt')
        torch.save(test_labels, current_folder + '/test_labels.pt')

            
        
# train_feats, train_labels = load_MAG_data_acled_general(glist, 0, 5, time_window, device)
# val_feats, val_labels = load_MAG_data_acled_general(glist, 5, 7, time_window, device)
# test_feats, test_labels = load_MAG_data_acled_general(glist, 7, 10, time_window, device)


# os.makedirs(folder_path, exist_ok=True)  # The `exist_ok` avoids error if the directory exists

# torch.save(train_feats, folder_path + '/train_feats.pt')
# torch.save(train_labels, folder_path + '/train_labels.pt')
# torch.save(val_feats, folder_path + '/val_feats.pt')
# torch.save(val_labels, folder_path + '/val_labels.pt')
# torch.save(test_feats, folder_path + '/test_feats.pt')
# torch.save(test_labels, folder_path + '/test_labels.pt')

1997 2000 2001 2004 2005 2024
creating splits train_0-3_valid_4-7_test_8-27
generating feats and labels 


: 

In [4]:
folder_path = 'splits/time_window_1/train_0-3_valid_4-7_test_8-27'

In [5]:
train_feats, train_labels, val_feats, val_labels, test_feats, test_labels = torch.load(f'{folder_path}/train_feats.pt'), torch.load(f'{folder_path}/train_labels.pt'), torch.load(f'{folder_path}/val_feats.pt'), torch.load(f'{folder_path}/val_labels.pt'), torch.load(f'{folder_path}/test_feats.pt'), torch.load(f'{folder_path}/test_labels.pt')


/tmp/ipykernel_2785515/2441314985.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_feats, train_labels, val_feats, val_labels, test_feats, test_labels = torch.load(

In [138]:
val_feats

[Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('action', 'involved_in_r_t0', 'actor'): 559, ('actor', 'involved_in_t0', 'action'): 559, ('actor', 'involved_with_r_t0', 'actor'): 594, ('actor', 'involved_with_t0', 'actor'): 594},
       metagraph=[('action', 'actor', 'involved_in_r_t0'), ('actor', 'action', 'involved_in_t0'), ('actor', 'actor', 'involved_with_r_t0'), ('actor', 'actor', 'involved_with_t0')])]

In [12]:
import pickle

# Save the data
with open("splits/train_1997_1999/val_2000_test_2001.pkl", "wb") as f:
    pickle.dump((train_feats, train_labels, val_feats, val_labels, test_feats, test_labels), f)

In [127]:
len(train_feats[0].nodes['actor'].data.keys())

2

In [18]:
# glist, label_dict = load_graphs('data/ogbn_graphs.bin')
# glist, label_dict = load_graphs('graph_data_acled.bin')
# device = torch.device('cuda:0')
# time_window = 2
import model.model
import importlib
importlib.reload(model.model)
from model.model import HTGNN, LinkPredictor
model_out_path = 'output/acled-MAG'
import pdb

import utils.utils
importlib.reload(utils.utils)
from utils.utils import compute_metric, compute_loss


auc_list, ap_list  = [], []
time_window=2



def initialize_model(time_window, graph_atom):
    device = torch.device('cuda:0')
    htgnn = HTGNN(graph=graph_atom, n_inp=128, n_hid=32, n_layers=2, n_heads=1, 
                time_window=time_window, norm=True, device=device)
    predictor = LinkPredictor(n_inp=32, n_classes=1)
    new_model = nn.Sequential(htgnn, predictor).to(device)
    return new_model

# train_feats, train_labels, val_feats, val_labels, test_feats, test_labels = load_MAG_data_acled(glist[:5], time_window, device)

def train_eval_model(time_window, train_feats, train_labels, val_feats, val_labels, test_feats, test_labels):
    f1_list_val = []
    f1_list_test = []
    device = torch.device('cuda:0')
    for k in range(1):
        # model = initialize_model(len(train_feats[0].nodes['actor'].data.keys()))
        model = initialize_model(time_window, test_feats[0])
        print(f'---------------Repeat time: {k+1}---------------------')
        print(f'# params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')
        optim = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=5e-4)

        early_stopping = EarlyStopping(patience=50, verbose=True, path=f'{model_out_path}/checkpoint_HTGNN_{k}.pt')
        for epoch in tqdm(range(10)):
        # for epoch in tqdm(range(500)):
            model.train()
            for (G_feat, (pos_label, neg_label)) in zip(train_feats, train_labels):

                G_feat = G_feat.to(device)

                pos_label = pos_label.to(device)
                neg_label = neg_label.to(device)
                
                # h = model[0](G_feat, 'author')
                h = model[0](G_feat, 'actor')
                pos_score = model[1](pos_label, h)
                neg_score = model[1](neg_label, h)

                loss = compute_loss(pos_score, neg_score, device)
                auc, ap, acc, f1 = compute_metric(pos_score, neg_score)
                
                # if epoch==4:
                #     pdb.set_trace()

                optim.zero_grad()
                loss.backward()
                optim.step()
            

            auc, ap, auc_mean, ap_mean, acc_mean, f1_mean = evaluate(model, val_feats, val_labels)
            f1_list_val.append(f1_mean)

            auc, ap, auc_mean, ap_mean, acc_mean, f1_mean = evaluate(model, test_feats, test_labels)
            f1_list_test.append(f1_mean)

            early_stopping(loss, model)

            # model = initialize_model(len(val_feats[0].nodes['actor'].data.keys()))
            # auc, ap = evaluate(model, val_feats, val_labels)

            if early_stopping.early_stop:
                print("Early stopping")
                break

        # model = initialize_model(len(test_feats[0].nodes['actor'].data.keys()))
        # auc, ap, auc_mean, ap_mean, acc_mean, f1_mean = evaluate(model, test_feats, test_labels)

        print(f'auc: {auc}, ap: {ap}')
        print(f'auc mean: {auc_mean}, ap mean: {ap_mean}, acc mean: {acc_mean}, f1 mean: {f1_mean} ')
        auc_list.append(auc)
        ap_list.append(ap)
    return f1_list_val, f1_list_test

    

Run the model for each of the splits

In [19]:
import os
root_path = "/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2"
for dir_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, dir_name)
    train_feats, train_labels, val_feats, val_labels, test_feats, test_labels = torch.load(f'{folder_path}/train_feats.pt'), torch.load(f'{folder_path}/train_labels.pt'), torch.load(f'{folder_path}/val_feats.pt'), torch.load(f'{folder_path}/val_labels.pt'), torch.load(f'{folder_path}/test_feats.pt'), torch.load(f'{folder_path}/test_labels.pt')
    f1_list_val, f1_list_test = train_eval_model(2, train_feats, train_labels, val_feats, val_labels, test_feats, test_labels)
    with open(f"{folder_path}/f1_val_scores.txt", "w") as file:
        for score in f1_list_val:
            file.write(f"{score}\n")
    with open(f"{folder_path}/f1_test_scores.txt", "w") as file:
        for score in f1_list_test:
            file.write(f"{score}\n")


/tmp/ipykernel_3473367/3542105101.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_feats, train_labels, val_feats, val_labels, test_feats, test_labels = torch.load(

---------------Repeat time: 1---------------------
# params: 57861


  0%|          | 0/10 [00:00<?, ?it/s]

0.037037037037037035
0.6666666666666666
0.0
0.0
0.6282722513089005
0.9420289855072463
0.6776315789473685
0.9045226130653267
0.9317738791423001
0.9333333333333333
0.9196261682242991
0.6837242359630419
0.7702805155420773
0.8159057437407953
0.8216650898770104
0.819954128440367
0.6801593625498008
0.7390865222758388
0.6682049276495894
0.7528136723634847
0.5581582902599069


 10%|█         | 1/10 [00:01<00:15,  1.76s/it]

...

0.6727095056777976
0.7134778293285322
Validation loss decreased (0.197707 --> 0.188427).  Saving model ...
0.9471153846153846
0.9708029197080292
0.9428571428571428
0.9098039215686274
0.9523809523809523
0.8558352402745996
0.9077306733167082
0.9449715370018975
0.9580152671755725
0.9039145907473309
0.6946229286438959
0.7619387027797576
0.8319571524213345
0.8238661382650815
0.8219032429558746
0.6791834301892415
0.7360284132455865
0.6498680602266259
0.6548442602802261
0.7133454698221537
0.516122828386435
0.5194515363053686
0.6727095056777976
0.7134778293285322
Validation loss decreased (0.188427 --> 0.182729).  Saving model ...


 90%|█████████ | 9/10 [00:11<00:01,  1.24s/it]

0.9471153846153846
0.9708029197080292
0.9428571428571428
0.9098039215686274
0.9523809523809523
0.8558352402745996
0.9077306733167082
0.9449715370018975
0.9580152671755725
0.9039145907473309
0.6946229286438959
0.7619387027797576
0.8319571524213345
0.8238661382650815
0.8219032429558746
0.6791834301892415
0.7360284132455865
0.6498680602266259
0.6548442602802261
0.7133454698221537
0.516122828386435
0.5194515363053686
0.6727095056777976
0.7134778293285322
Validation loss decreased (0.182729 --> 0.182408).  Saving model ...


100%|██████████| 10/10 [00:12<00:00,  1.26s/it]

auc: 0.7417152535806872, ap: 0.726798931876052
auc mean: 0.7425412494700531, ap mean: 0.7638906715534944, acc mean: 0.6866133969764222, f1 mean: 0.6992371070377079 


In [27]:
grouped_folders = {}


root_path = "/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2"
for dir_name in os.listdir(root_path):
    prefix = "_".join(dir_name.split("_")[:2])
    folder_path = os.path.join(root_path, dir_name)
    if prefix not in grouped_folders:
        grouped_folders[prefix] = []
    grouped_folders[prefix].append(folder_path)
    
grouped_folders        
        

{'train_0-12': ['/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-12_valid_12-20_test_20-27',
  '/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-12_valid_12-16_test_16-27',
  '/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-12_valid_12-24_test_24-27'],
 'train_0-8': ['/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-8_valid_8-24_test_24-27',
  '/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-8_valid_8-16_test_16-27',
  '/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-8_valid_8-20_test_20-27',
  '/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-8_valid_8-12_test_12-27'],
 'train_0-20': ['/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-20_valid_20-24_test_24-27'],
 'train_0-16': ['/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-16_valid_16-20_test_20-27',
  '/home/niamatzawad/niamatzawad/HTGNN/splits/time_window_2/train_0-16_valid_16-24_tes

In [41]:
import matplotlib.pyplot as plt

def get_plot(folder, train_start, train_end, val_start, val_end, test_start, test_end, scores):
    plt.figure(figsize=(10,6))
    x = range(11)
    plt.ylim(0, 1)  # Explicitly setting y-axis from 0 to 1
    plt.yticks([i * 0.1 for i in range(11)])  # Ticks from 0 to 1 at 0.1 intervals

    plt.xticks(range(min(x), max(x) + 1, 1))
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.title('F1 scores(Validation)')

    # Adding a grid for better readability
    plt.grid(True)

    # labels = validation_splits
    colors = ['red', 'green', 'blue', 'cyan', 'black', 'gray', 'purple', 'orange']
    # colors = ['r','b']
    for i,v_split in enumerate(scores):
        plt.plot(v_split, color=colors[i],  linestyle='-', linewidth=2)
    # plt.plot(all_val_f1_scores[0], color=colors[0], label=labels[0], linestyle='-', linewidth=2)
    plt.legend(loc='lower right')
    plt.savefig(folder + "/validation.png")  # You can specify the file path here
    # plt.show()

In [43]:
import re

def get_scores(file_path):
    scores = []
    with open(file_path, 'r') as file:
        for line in file:
            scores.append(float(line.strip()))
    return scores

for key,item in grouped_folders.items():
    val_scores = []
    test_scores = []
    labels = []
    
    train_start, train_end = dict_ind_year[0], dict_ind_year[int(key.split("-")[1] - 1)]
    
    for folder in item:
        folder_name = folder.split("/")[-1]
        pattern = r'(\d+)-(\d+)'
        matches = re.findall(pattern, folder_name)
        results = {
            "train": tuple(map(int, matches[0])),
            "valid": tuple(map(int, matches[1])),
            "test": tuple(map(int, matches[2]))
        }
        print(results['train'], results['valid'], results['test'])
        labels.append(f"validation({dict_ind_year[results['valid'][0]]} - {dict_ind_year[results['valid'][1]]})")
        val_scores.append(get_scores(os.path.join(folder, 'f1_val_scores.txt')))
        test_scores.append(get_scores(os.path.join(folder, 'f1_test_scores.txt')))
    get_plot(folder, train_start, train_end, labels, val_scores)
        

(0, 12) (12, 20) (20, 27)
(0, 12) (12, 16) (16, 27)
(0, 12) (12, 24) (24, 27)
(0, 8) (8, 24) (24, 27)
(0, 8) (8, 16) (16, 27)
(0, 8) (8, 20) (20, 27)
(0, 8) (8, 12) (12, 27)
(0, 20) (20, 24) (24, 27)
(0, 16) (16, 20) (20, 27)
(0, 16) (16, 24) (24, 27)
(0, 4) (4, 8) (8, 27)
(0, 4) (4, 16) (16, 27)
(0, 4) (4, 24) (24, 27)
(0, 4) (4, 20) (20, 27)
(0, 4) (4, 12) (12, 27)


In [147]:
test_feats

[Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('action', 'involved_in_r_t0', 'actor'): 506, ('actor', 'involved_in_t0', 'action'): 506, ('actor', 'involved_with_r_t0', 'actor'): 584, ('actor', 'involved_with_t0', 'actor'): 584},
       metagraph=[('action', 'actor', 'involved_in_r_t0'), ('actor', 'action', 'involved_in_t0'), ('actor', 'actor', 'involved_with_r_t0'), ('actor', 'actor', 'involved_with_t0')]),
 Graph(num_nodes={'action': 25, 'actor': 18622},
       num_edges={('action', 'involved_in_r_t0', 'actor'): 526, ('actor', 'involved_in_t0', 'action'): 526, ('actor', 'involved_with_r_t0', 'actor'): 591, ('actor', 'involved_with_t0', 'actor'): 591},
       metagraph=[('action', 'actor', 'involved_in_r_t0'), ('actor', 'action', 'involved_in_t0'), ('actor', 'actor', 'involved_with_r_t0'), ('actor', 'actor', 'involved_with_t0')])]

In [ ]:
import matplotlib.pyplot as plt

# Plotting
plt.figure(figsize=(10,6))
x = range(11)
plt.ylim(0, 1)  # Explicitly setting y-axis from 0 to 1
plt.yticks([i * 0.1 for i in range(11)])  # Ticks from 0 to 1 at 0.1 intervals

plt.xticks(range(min(x), max(x) + 1, 1))
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('F1 scores(Validation)')

# Adding a grid for better readability
plt.grid(True)

# Show legend to label each line


# labels = validation_splits
colors = ['red', 'green', 'blue', 'cyan', 'black', 'gray', 'purple', 'orange']
# colors = ['r','b']
for i,v_split in enumerate(all_val_f1_scores):
    plt.plot(v_split, color=colors[i],  linestyle='-', linewidth=2)
# plt.plot(all_val_f1_scores[0], color=colors[0], label=labels[0], linestyle='-', linewidth=2)
plt.legend(loc='lower right')
plt.savefig(root_path + "/validation.png")  # You can specify the file path here
plt.show()